In [44]:

# file_name	scenescore_1	score_2	score_3	score_4	score_5	score_6

In [51]:
from dataclasses import dataclass
import natsort
import pathlib
from typing import List
import cv2
from numpy.core.multiarray import ndarray
from scene import Screen, TitleScreen, MarioGameScreen, RadRacerGameScreen, TetrisGameScreen, ScoreScreen

In [46]:
samples_paths = pathlib.Path('samples').glob('*.png')
non_yt_sample_paths = [sample for sample in samples_paths if 'yt' not in str(sample)]

print(non_yt_sample_paths)

[WindowsPath('samples/snap_10142.png'), WindowsPath('samples/snap_10452.png'), WindowsPath('samples/snap_10683.png'), WindowsPath('samples/snap_1079.png'), WindowsPath('samples/snap_11118.png'), WindowsPath('samples/snap_120.png'), WindowsPath('samples/snap_12417.png'), WindowsPath('samples/snap_13042.png'), WindowsPath('samples/snap_14042.png'), WindowsPath('samples/snap_17935.png'), WindowsPath('samples/snap_19519.png'), WindowsPath('samples/snap_20038.png'), WindowsPath('samples/snap_2078.png'), WindowsPath('samples/snap_2387.png'), WindowsPath('samples/snap_24277.png'), WindowsPath('samples/snap_24356.png'), WindowsPath('samples/snap_24523.png'), WindowsPath('samples/snap_2576.png'), WindowsPath('samples/snap_2791.png'), WindowsPath('samples/snap_2969.png'), WindowsPath('samples/snap_3177.png'), WindowsPath('samples/snap_3267.png'), WindowsPath('samples/snap_3490.png'), WindowsPath('samples/snap_3558.png'), WindowsPath('samples/snap_3644.png'), WindowsPath('samples/snap_3898.png'),

In [52]:
screens: List[Screen] = [
    TitleScreen(),
    MarioGameScreen(),
    RadRacerGameScreen(),
    TetrisGameScreen(),
]

@dataclass
class TrainingCsvEntry:
    file_name: str
    scene_name: str
    score_0: str="n/a"
    score_1: str="n/a"
    score_2: str="n/a"
    score_3: str="n/a"
    score_4: str="n/a"
    score_5: str="n/a"

def process_path(path: pathlib.Path):

    hsv_image = cv2.imread(str(path))

    frame = cv2.resize(hsv_image, (256, 224))

    for screen in screens:
        hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        if screen.match(hsv_image):
            if isinstance(screen, ScoreScreen):
                score = screen.score(hsv_image)
                score_list = list(f'{score:06}')
                return TrainingCsvEntry(
                    file_name= str(path.stem),
                    scene_name = "screen.__class__.__name__",
                    score_0=score_list[0],
                    score_1=score_list[1],
                    score_2=score_list[2],
                    score_3=score_list[3],
                    score_4=score_list[4],
                    score_5=score_list[5],
                )
    return TrainingCsvEntry(
        file_name=str(path.stem),
        scene_name="n/a",
    )


csv_data = [process_path(sample_path) for sample_path in natsort.os_sorted(non_yt_sample_paths)]
